In [1]:
import pandas as pd

import os

In [2]:
all_rows = pd.DataFrame(columns = ['id', 'link', 'name', 'rate', 'description', 'reviews'])
all_rows

,id,link,name,rate,description,reviews


In [3]:
for name in os.listdir():
    if name.endswith('.csv') and name.startswith('rows'):
        temp_df = pd.read_csv(name)
        all_rows = pd.concat([all_rows, temp_df])

In [4]:
all_rows.shape

(8685, 6)

In [19]:
all_rows.to_parquet('all_rows.pqt', index=False)

In [20]:
pd.read_parquet('all_rows.pqt')

,id,link,name,rate,description,reviews
0,451006,https://www.labirint.ru/books/451006/,Роберт Стивенсон: Остров Сокровищ,9.38,Знаменитый роман Роберта Льюиса Стивенсона в ...,Есть у меня одна страсть. По нынешним временам...
1,786953,https://www.labirint.ru/books/786953/,Жюль Верн: Вокруг света в восемьдесят дней,8.26,"""Вокруг книги"" - это лучшие произведения миров...",Позволю себе не разделить восторга предыдущих ...
2,840898,https://www.labirint.ru/books/840898/,"Тур Хейердал: Экспедиция ""Кон-Тики""",8.33,"Отважный человек, нестандартно мыслящий, самоо...","Наверное, впервые вижу книгу, про которую хоче..."
3,614463,https://www.labirint.ru/books/614463/,Артур Дойл: Приключения Шерлока Холмса,9.06,Интерактивное комментированное издание рассказ...,Фотографии сделаны на выставке. Фотографировал...
4,869738,https://www.labirint.ru/books/869738/,Артур Дойл: Записки о Шерлоке Холмсе,9.07,В книгу вошли пять рассказов Артура Конан Дойл...,"Вторая книга, наконец-то! Уже и не чаяли, как ..."
...,...,...,...,...,...,...
8680,959811,https://www.labirint.ru/books/959811/,Фрэнк Герберт: Термитник Хеллстрома,10.00,"Уединенная безлюдная ферма, затерявшаяся между...",None
8681,875877,https://www.labirint.ru/books/875877/,Михаил Филин: Мария Волконская. «Утаенная любо...,10.00,Пленительный образ княгини Марии Николаевны Во...,None
8682,824430,https://www.labirint.ru/books/824430/,Маркиз де Сад: Преступления любви,10.00,"Каким является человек, живущий идеей абсолютн...","смысл рассказов, вошедших в данное издание, по..."
8683,724758,https://www.labirint.ru/books/724758/,Эрих Ремарк: Черный обелиск,9.25,"Их называли ""потерянным поколением"". И они дей...","""У каждого своя смерть, он должен пережить её ..."


In [24]:
os.path.exists(os.path.join('.', 'tfidf_vectors.npy'))

True

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer


vectorizer = TfidfVectorizer()

tfidf_vectors = vectorizer.fit_transform(all_rows.name)

In [9]:
tfidf_vectors.shape

(8685, 20283)

In [12]:
from sklearn.metrics.pairwise import euclidean_distances, cosine_similarity

import numpy as np

In [29]:
# Новое предложение
new_sentence = '''Сенека
Наставник императора'''

new_tfidf_vector = vectorizer.transform([new_sentence])

new_tfidf_vector_array = new_tfidf_vector.toarray()

euclidean_dist = cosine_similarity(new_tfidf_vector_array.reshape(1, -1), tfidf_vectors)

all_rows.iloc[np.argmax(euclidean_dist)]

ValueError: setting an array element with a sequence.

In [88]:
tfidf_vectors.shape

(7229, 17140)

In [90]:
euclidean_dist = euclidean_distances(new_tfidf_vector_array.reshape(1, -1), tfidf_vectors)

In [91]:
import numpy as np

np.argmin(euclidean_dist)

3491

In [93]:
all_rows.iloc[3491]

id                                                        493711
link                       https://www.labirint.ru/books/493711/
name                                      Олег Михайлов: Кутузов
rate                                                         0.0
description    Исторический роман посвящен Михаилу Илларионов...
reviews                                                      NaN
Name: 908, dtype: object

In [72]:
import requests


def ocr_space_file(filename, overlay=False, api_key='helloworld', language='eng'):
    """ OCR.space API request with local file.
        Python3.5 - not tested on 2.7
    :param filename: Your file path & name.
    :param overlay: Is OCR.space overlay required in your response.
                    Defaults to False.
    :param api_key: OCR.space API key.
                    Defaults to 'helloworld'.
    :param language: Language code to be used in OCR.
                    List of available language codes can be found on https://ocr.space/OCRAPI
                    Defaults to 'en'.
    :return: Result in JSON format.
    """

    payload = {'isOverlayRequired': overlay,
               'apikey': api_key,
               'language': language,
               }
    with open(filename, 'rb') as f:
        r = requests.post('https://api.ocr.space/parse/image',
                          files={filename: f},
                          data=payload,
                          )
    return r.json()

In [73]:
res = ocr_space_file('test.png', api_key='K81646083088957', language='rus')

In [80]:
res["ParsedResults"][0]["ParsedText"].strip()

'с.михплков\r\nи ск.зни'

In [70]:
eval(res)

NameError: name 'false' is not defined

In [ ]:
["ParsedText']